In [1]:
#   ==========   #
#%% PARAMETERS %%#
#   ==========   #

orbital = "A1"
norbitals = 1 

# construction and diagonalization in a single step
onestep = true
# use precomputed spin-CG coefficients
spinarray = true
# parallel computation
distributed = parallel = true
# parallel method: distfor or async
method = "distfor"
# discretization ("standard" or "co2005")
discretization = "co2005"
# calculation 
calculation = "IMP"

z = 0.0
if length(ARGS)>0
    # clean/imp
    calculation = "IMP"
    if ARGS[1]=="CLEAN" 
        calculation = "CLEAN"
    end
    # twisting parameter
    if length(ARGS)>1
        z = parse(Float64,ARGS[2])
    end
end

# Free orbital --> Local moment ==> Strong coupling
eps = -0.1
cou =  0.2
#gam =  0.01
gam = 0.01

L = 3.0
betabar = 1.0

cutoff_type = "multiplet" # multiplet/energy
cutoff_magnitude = 20

iterations = 35

spinarray = true  # faster
max_spin2 = 6

distworkers = 6

println( "================" )
println( "SETUP PARAMETERS" )
println( "================" )
@show onestep 
@show distributed 
@show calculation 
@show method 
@show z 
@show eps 
@show cou 
@show gam 
@show L 
@show betabar 
@show cutoff_type 
@show cutoff_magnitude 
@show iterations 
@show spinarray 
@show max_spin2 
@show distworkers


#   =======   #
#%% MODULES %%#
#   =======   #

using Distributed 


using BenchmarkTools 
using DelimitedFiles
using ProfileVega
using Profile

include( "modules/symbols.jl" )
include( "modules/numericals.jl" )
include( "modules/compoundoperators.jl" )
include( "modules/shell.jl" )
include( "modules/thermo.jl" )
include( "modules//spectral.jl" )

if distributed 

    # kill current processes
    for i in workers()
        t = rmprocs(i, waitfor=0)
        wait(t)
    end

    # add requested workers
    if distworkers ≥ nprocs()
        addprocs(distworkers)
    else 
        println( "more workers than processors!" )
    end

    @everywhere begin 
        using ProgressMeter
        using PartialWaveFunctions
        using StaticArrays
        include( "modules/symmetry.jl" )
        include( "modules/diagonalization.jl" )
    end

    println( "DISTRIBUTED CALCULATION WITH $(nworkers()) WORKERS" )

else

    using ProgressMeter
    using PartialWaveFunctions
    using StaticArrays
    include( "modules/symmetry.jl" ) 
    include( "modules/diagonalization.jl" )

    println( "SERIAL CALCULATION" )

end
println()

SETUP PARAMETERS
onestep = true
distributed = true
calculation = "IMP"
method = "distfor"
z = 0.0
eps = -0.1
cou = 0.2
gam = 0.01
L = 3.0
betabar = 1.0
cutoff_type = "multiplet"
cutoff_magnitude = 20
iterations = 35
spinarray = true
max_spin2 = 6
distworkers = 6


┌ Warning: rmprocs: process 1 not removed
└ @ Distributed /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.6/Distributed/src/cluster.jl:1038


      From worker 5:	WARNING: using LinearAlgebra.I in module Main conflicts with an existing identifier.


      From worker 3:	WARNING: using LinearAlgebra.I in module Main conflicts with an existing identifier.
      From worker 6:	WARNING: using LinearAlgebra.I in module Main conflicts with an existing identifier.
      From worker 4:	WARNING: using LinearAlgebra.I in module Main conflicts with an existing identifier.


      From worker 7:	WARNING: using LinearAlgebra.I in module Main conflicts with an existing identifier.
      From worker 2:	WARNING: using LinearAlgebra.I in module Main conflicts with an existing identifier.


DISTRIBUTED CALCULATION WITH 6 WORKERS


In [2]:
#   ======================   #
#%% ORBITAL CLEBSCH-GORDAN %%#
#   ======================   #

CG_PATH = "/home/aitor/Bulegoa/ClebschGordan/C4v/cg_symbolic/"
ASYM_PATH = "/home/aitor/Bulegoa/AntiSymmetricPart/C4v/";

ORBITAL_IRREPS = ( "A1" ,)

oirreps = cg_shortcircuit( CG_PATH , ORBITAL_IRREPS... )
oirreps2indices = Dict( o=>i for (i,o) in enumerate(oirreps) )
oirreps2dimensions = Dict( "A1" => 1 ) 
oindex2dimensions = collect( oirreps2dimensions[I] for I in oirreps )

cg_o_full = get_cg_o_fulldict( oirreps , CG_PATH )
cg_o_fullmat = get_cg_o_fullmat( cg_o_full )
cg_o_fullmatint = get_cg_o_fullmatint( cg_o_full , oirreps )


#   ===================   #
#%% SPIN CLEBSCH-GORDAN %%#
#   ===================   #
cg_s_fullmatint = get_cg_s_fullmatint( max_spin2 );

In [4]:

#   ===========================   #
#&& SYMBOLIC CALCULATION OF N=0 &&#
#   ===========================   #

#   ------------ #
#%% state tuples # 
#   ------------ #
a_1 = ( 0 , "a" , 1 )
a_1u = ( a_1... , "u" )
a_1d = ( a_1... , "d" )

tuples_0 = [ a_1u , a_1d ]

#   ----- #
#%% basis #
#   ----- #
hilbert_0 = HilbertSpace( tuples_0... )
basis_0 = CanonicalBasis( hilbert_0 )
println( "HILBERT SPACE" )
println( hilbert_0 )
println()
println( "BASIS" )
println( basis_0 )


#   --------- #
#%% symstates #
#   --------- #
hiztegia = Dict( 
    "a" =>  "A1",
    "u" =>  0.5 ,
    "d" => -0.5
)

symstates_0 = oneirrep_symstates( hilbert_0 , hiztegia , "A1" , "$(ASYM_PATH)A1/" )
symstates_0_new = Dict()
for (q,s) in symstates_0 
    push!( symstates_0_new , (q[1:5]...,1)=>s ) 
end
symstates_0 = symstates_0_new
println( "SYMSTATES" )
print_dict( symstates_0 )

irrepsubs_0 = get_multiplets( symstates_0 )
println( "MULTIPLETS" )
for m in irrepsubs_0 
    println( m )
end
println()

#   --------- #
#%% operators #
#   --------- #
epsilon_symparams = Dict( 
    "A1" => eps
)
epsilon = epsilon_sym( symstates_0 , epsilon_symparams ; verbose=false )
println( "EPSILON OPERATOR" )
print_diagonals( epsilon )

u_symparams = Dict( 
    ("A1",0) => [cou][:,:]
)
coulomb = u_sym( symstates_0 , u_symparams ; verbose=false );
println( "COULOMB OPERATOR" )
print_diagonals( coulomb )

#   ---------------- #
#%% full hamiltonian #
#   ---------------- #
H = epsilon + coulomb 
# α = ϵ_0^z (scaling factor)
if discretization=="standard"
    global α = 0.5 * L^z * (1+L^-1)
elseif discretization=="co2005"
    global α = compute_ebar0_z(z,L;discretization=discretization)
end
H.matrix ./= α
gam = sqrt(2.0*gam/pi) / α # for later


#   ----------------------- #
#%% irrEU for IMP and CLEAN #
#   ----------------------- #
multiplets_0 = get_multiplets( symstates_0 )
irreps_0 = get_irreps( symstates_0 )
oirreps2dimensions = Dict( "A1" => 1 ) 

#%% block diagonalization of imp
if calculation=="IMP"
    global irrEU_imp = Dict{ Tuple{Int64,String,Float64} , Tuple{Vector{Float64},Matrix{ComplexF64}} }()
    for G in irreps_0 
        size = convert( Int64 , oirreps2dimensions[G[2]] * (2*G[3]+1) )
        Gstates = get_irrepstates_onepartner( symstates_0 , G ; partner="o1_smax" )
        N = length(Gstates)
        hmat = Matrix{ComplexF64}( undef , N , N )
        for r1=1:N, r2=1:N
            s1 = Gstates[( G... , 1 , G[3] , r1 )]
            s2 = Gstates[( G... , 1 , G[3] , r2 )]
            hmat[r1,r2] = s1 * H * s2
        end
        hmat = 0.5*(hmat + conj(hmat'))
        (e,u) = eigen( hmat )
        push!( irrEU_imp , G=>(e,u) )
    end
    #println( "COMPARISON OF IMP DIAGONALIZATION" ) e_block = collect( e for (G,(E,U)) in irrEU_imp for e in E )
    #sort!( e_block )
    #e_noblock = sort( e )
    #@show e_block 
    #@show e_noblock
    println( "IMPURITY irrEU" )
    for (G,(E,U)) in irrEU_imp
        @show G 
        println( "size = $(convert( Int64 , oirreps2dimensions[G[2]] * (2*G[3]+1) ))")
        @show E 
        @show U
        println()
    end
elseif calculation=="CLEAN"
    global irrEU_clean = Dict{ Tuple{Int64,String,Float64} , Tuple{Vector{Float64},Matrix{ComplexF64}} }(
        (0,"A1",0.0) => ( Float64[0.0] , ComplexF64[0.0im][:,:] )
    )
    println( "CLEAN irrEU" )
    print_irrEU( irrEU_clean , oirreps2dimensions , oirreps )
end


#   ==============================================   #
#&& NUMERIC ADDITION OF INNERMOST CONDUCTION SHELL &&#
#   ==============================================   #

#   ------------ #
#%% irrEU choice #
#   ------------ #
println( "irrEU choice" )
if calculation=="CLEAN" 
    irrEU = irrEU_clean
elseif calculation=="IMP"
    irrEU = irrEU_imp
end
# conversion to int keys
irrEU = Dict( 
    (N,oirreps2indices[I],convert(Int64,2*S)) => X 
    for ((N,I,S),X) in irrEU 
)
print_irrEU( irrEU , oirreps2dimensions , oirreps )


#   --------------- #
#%% combinations u' #
#   --------------- #
print( "COMBINATIONS U' " )
#combinations_uprima = Dict{ Tuple{Int64,String,Float64,Int64} , NTuple{2,Tuple{Int64,String,Float64,Int64}} }()
combinations_uprima = Dict{ Tuple{Int64,Int64,Int64,Int64} , NTuple{2,Tuple{Int64,Int64,Int64,Int64}} }()
#m_vac = (0,"A1",0.0,1)
m_vac = (0,1,0,1)
if calculation=="IMP"
    println( "for IMP")
    for m_mu in get_multiplets( symstates_0 )
        mint_mu = convert_to_int( m_mu , oirreps2indices )
        push!( combinations_uprima , mint_mu=>(mint_mu,m_vac) )
    end 
elseif calculation=="CLEAN" 
    println( "for CLEAN" )
    push!( combinations_uprima , m_vac=>(m_vac,m_vac) )
end
#print_dict( combinations_uprima )

irreps_uprima = Set( k[1:3] for k in keys(combinations_uprima) )
combinations_uprima = 
        Dict{ NTuple{3,Int64} , 
              Vector{NTuple{3,NTuple{4,Int64}}} }(
            G => NTuple{3,NTuple{4,Int64}}[
                (m_u,m_mu,m_i)
                for (m_u,(m_mu,m_i)) in combinations_uprima 
                if m_u[1:3]==G
                ]
            for G in irreps_uprima
        )
print_dict( combinations_uprima )

#   ---------------------- #
#%% pseudo-cg coefficients #
#   ---------------------- #
pcg = get_pseudoCG( symstates_0 , 
                    basis_0 , 
                    hiztegia ,
                    oirreps2dimensions ; 
                    verbose=false )
println( "PSEUDO-CG COEFFICIENTS" )
println()
print_dict( pcg )

pcgmat = Dict{ NTuple{3,NTuple{3,Int64}} , Array{ComplexF64,9} }()
shell_irrmult = get_irreps( multiplets_0 ; multiplicity=true )
pcgmat= get_pseudoCG_mat( pcg , 
                          shell_irrmult , 
                          oindex2dimensions , 
                          oirreps2indices );

#   ------------------- #
#%% shell cops and qq_a #
#   ------------------- #
shell_cops = shell_coperators( basis_0 , hiztegia )
# CONVERT TO NEW FORMAT 
qq_a = collect( convert_to_int(k,oirreps2indices) 
            for k in keys(shell_cops) )
println( "INGREDIENT 5: qq_a" )
for q_a in qq_a
    println( q_a )
end

#   -----------------   #
#%% excitation matrix %%#
#   -----------------   #
M = get_M0( pcg , qq_a )
println( "===================" )
println( "M excitation matrix" )
println( "===================" )
print_dict( M )
println()


#   ------------------ #
#%% hopping parameters #
#   ------------------ #
#hop = Dict( ("A1",1) => gam )
hop = Dict( (1,1) => ComplexF64(gam) )
#println( "HOPPING PARAMETERS" )
#@show hop 
#println()

#   -------------------------- #
#%% block and shell multiplets #
#   -------------------------- #
if calculation=="IMP"
    symstates_block = symstates_0
    multiplets_block = get_multiplets( symstates_block )
elseif calculation=="CLEAN"
    multiplets_block = Set([(0,"A1",0.0,1)])
end
symstates_shell = symstates_0
multiplets_shell = get_multiplets( symstates_shell )

# convert to int
multiplets_block = Set( convert_to_int(m,oirreps2indices)
                        for m in multiplets_block )
multiplets_shell = Set( convert_to_int(m,oirreps2indices)
                        for m in multiplets_shell )

#println( "BLOCK MULTIPLETS" )
#for m in multiplets_block 
#    println( m )
#end
#println()
#println( "SHELL MULTIPLETS" )
#for m in multiplets_shell 
#    println( m )
#end

#   ------------------------   #
#%% impurity quantum numbers %%#
#   ------------------------   #
omults = ordered_multiplets(multiplets_block)
mult2index = Dict( m=>i for (i,m) in 
                   enumerate(omults))
mm_i = Dict( 
            m=>[(i==mult2index[m] ? 1.0 : 0.0)
                for i in 1:length(multiplets_block)] 
                for m in omults
           )
#@show mm_i 
m_imp = mult_thermo( irrEU ,
                     betabar ,
                     oindex2dimensions ,
                     mm_i )

#   --------------------------------------- #
#%% matrix construction and diagonalization #
#   --------------------------------------- #
if distributed 
    @time begin
    global (irrEU,combinations_uprima) = matdiag_distributed_mat( 
        multiplets_block , 
        multiplets_shell ,
        irrEU , 
        hop , 
        cg_o_fullmatint , 
        cg_s_fullmatint ,
        pcg , 
        pcgmat, 
        qq_a , 
        combinations_uprima , 
        oindex2dimensions ;
        verbose=false );
    end
else
    @time begin
    global (irrEU,combinations_uprima) = matdiag_serial( 
        multiplets_block , 
        multiplets_shell ,
        irrEU , 
        hop , 
        cg_o_fullmatint , 
        cg_s_fullmatint ,
        pcg , 
        pcgmat, 
        qq_a , 
        combinations_uprima , 
        oindex2dimensions ;
        verbose=false );
    end
end
minE = minimum([e for (E,U) in values(irrEU) for e in E])
irrEU = Dict( G=>(E.-minE,U) for (G,(E,U)) in irrEU )
energies = sort([e for (E,U) in values(irrEU) for e in E])

mm_i = imp_mults( irrEU ,
                  oindex2dimensions ,
                  combinations_uprima ,
                  mm_i )
m_imp = mult_thermo( irrEU ,
                     betabar ,
                     oindex2dimensions ,
                     mm_i )

#println( "irrEU after adding the first shell to $calculation" )
#println( "**********************************" )
#minE = minimum([e for (E,U) in values(irrEU) 
#                  for e in E ])
#for (G,(E,U)) in irrEU 
#    @show G 
#    @show E
#    irrEU[G] = ( E./sqrt(L) , U )
#    println()
#end
#println()

#   ~~~~~~~~~~~~~~~~~~~   #
#%% transformation of M %%#
#   ~~~~~~~~~~~~~~~~~~~   #
@show combinations_uprima

println( "===================" )
println( "TRANSFORMATION OF M" )
println( "===================" )
println()

# TODO 
# - inbounds all for loops
#
println( "M matrix" )
print_dict( M )
println()
q_a = qq_a[1]

M = get_new_M( M , 
               qq_a , 
               irrEU , 
               oindex2dimensions , 
               combinations_uprima ,
               cg_o_fullmatint , 
               cg_s_fullmatint )
A = get_A(M,irrEU)
AA = [A]




HILBERT SPACE


(0, "a", 1, "u")
(0, "a", 1, "d")

BASIS
1: 

|)
2: | 0 a 1 u )
3: | 0 a 1 d )
4: | 0 a 1 u , 0 a 1 d )
SYMSTATES


(1, "A1", 0.5, 1, 0.5, 1)
(1.0 + 0.0im) | 0 a 1 u )



(0, "A1", 0, 1, 0, 1)
(1.0 + 0.0im) |)

(

2, "A1", 0, 1, 0, 1)
(1.0 + 0.0im) | 0 a 1 u , 0 a 1 d )

(1, "A1", 0.5, 1, -0.5, 1)
(1.0 + 0.0im) | 0 a 1 d )

MULTIPLETS

In [ ]:

#   =============   #
#%% NRG PROCEDURE %%#
#   =============   #
hopchannels = collect(keys( hop ))
@profile nrg = NRG_mat_spectral( 
           iterations , 
           cutoff_type , 
           cutoff_magnitude , 
           L , 
           hopchannels , 
           irrEU , 
           multiplets_shell ,
           cg_o_fullmatint , 
           pcg ,
           pcgmat ,
           qq_a ,
           combinations_uprima ,
           betabar ,
           oindex2dimensions ,
           mm_i ,
           M ,
           AA ;
           spinarray=true ,
           cg_s_fullmatint=cg_s_fullmatint ,
           distributed=distributed ,
           z=z );
println()